In [1]:
import altair as alt
import pandas as pd
from pyproj import Transformer

In [2]:
url_geojson = 'SHlandkreise_simplify200.geojson'
data_geojson_remote = alt.Data(url=url_geojson, format=alt.DataFormat(property='features',type='json')) 

file_path = 'opendata_wka_ib_gv_vb_sh_20220701.csv'
wps = pd.read_csv(file_path, encoding='cp1250', sep=';')

print(wps.head())

    KREIS GEMEINDE                   TYP HERSTELLER NABENHOEHE  \
0    Kiel     Kiel  Nordex N131 - 3,0 MW     Nordex        134   
1    Kiel     Kiel  Nordex N131 - 3,0 MW     Nordex        134   
2    Kiel     Kiel  Nordex N131 - 3,0 MW     Nordex        134   
3  Lübeck   Lübeck        Enercon E 82 E    Enercon         78   
4  Lübeck   Lübeck        Enercon E-82 E    Enercon         85   

  ROTORDURCHMESSER SCHALLLEISTUNGSPEGEL  LEISTUNG        LEISTUNGSBEZUG  \
0              131                  NaN      3000  kW Nennleist. Rotor.   
1              131                  NaN      3000  kW Nennleist. Rotor.   
2              131                  NaN      3000  kW Nennleist. Rotor.   
3               82          104,5 dB(A)      2300  kW Nennleist. Rotor.   
4               82                104,5      2300  kW Nennleist. Rotor.   

    OSTWERT  NORDWERT GENEHMIGT_AM INBETRIEBNAHME        STATUS      BST_NR  \
0  32571605   6013660          NaN            NaN  im Gen.Verf.  20000003

In [3]:
transformer = Transformer.from_crs("EPSG:4647", "EPSG:4326", always_xy=True)
wps['longitude'], wps['latitude'] = transformer.transform(wps['OSTWERT'].values, wps['NORDWERT'].values)

In [4]:
background = alt.Chart(data_geojson_remote).mark_geoshape(
    fill='lightblue',
    stroke='white'
).properties(
    width=500,
    height=300
)

points = alt.Chart(wps).mark_circle(size=50).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color=alt.Color('LEISTUNG:Q',
        scale=alt.Scale(domain=[wps['LEISTUNG'].min(), wps['LEISTUNG'].max()],
                        range=['lightblue', 'blue']),  
        title='LEISTUNG'),
    tooltip=['ANL_NR', 'LEISTUNG']
)

In [5]:
map_chart = background + points
map_chart.show()

alt.LayerChart(...)